#### 1. Dependencies

In [1]:
import os

import nest_asyncio
from dotenv import load_dotenv
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.prompts import SystemMessagePromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.vectorstores import Qdrant
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from qdrant_client import AsyncQdrantClient, QdrantClient
from tqdm import tqdm


In [2]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

#### 2. Setup

In [3]:
load_dotenv()
nest_asyncio.apply()

In [4]:
MODEL = "gpt-4-turbo-preview"
EMBEDDING_MODEL = "text-embedding-3-large"
llm = ChatOpenAI(model=MODEL, temperature=0)
embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)
client = QdrantClient(url=os.getenv("QDRANT__URL"), api_key=os.getenv("QDRANT__API_KEY"), prefer_grpc=True)
aclient = AsyncQdrantClient(url=os.getenv("QDRANT__URL"), api_key=os.getenv("QDRANT__API_KEY"), prefer_grpc=True)

### 3. Testing Retrievers ~ Tools

In [5]:
qdrant_args = {
    "client": client,
    "async_client": aclient,
    "embeddings": embeddings,
}

#### 3.1 Retriever: Constitución Política de la República de Colombia

In [6]:
constitucion_vectorstore = Qdrant(collection_name="constitucion", **qdrant_args)
constitucion_retriever = constitucion_vectorstore.as_retriever(search_kwargs ={"k": 3})
pretty_print_docs(await constitucion_retriever.ainvoke("Colombia es un Estado social de derecho"))

Document 1:

Constitución Política de la República de Colombia 
<Esta versión corresponde a la segunda edición corregida de la Constitución Política de Colombia, publicada en la Gaceta Constitucional No. 116 de 20 de julio de 1991> 
PREÁMBULO
El pueblo de Colombia,
en ejercicio de su poder soberano, representado por sus delegatarios a la Asamblea Nacional Constituyente, invocando la protección de Dios, y con el fin de fortalecer la unidad de la Nación y asegurar a sus integrantes la vida, la convivencia, el trabajo, la justicia, la igualdad, el conocimiento, la libertad y la paz, dentro de un marco jurídico, democrático y participativo que garantice un orden político, económico y social justo, y comprometido a impulsar la integración de la comunidad latinoamericana, decreta, sanciona y promulga la siguiente:
Constitución Política de Colombia
Resumen de Notas de Vigencia

Jurisprudencia Concordante
Corte ConstitucionalSentencias de control de constitucionalidad:C-477-92; C-478-92; C-479

#### 3.2 Retriever: Código Penal - LEY 599 DE 2000

In [7]:
codigo_penal_vectorstore = Qdrant(collection_name="codigo_penal", **qdrant_args)
codigo_penal_retriever = codigo_penal_vectorstore.as_retriever(search_kwargs ={"k": 3})
pretty_print_docs(await codigo_penal_retriever.ainvoke("Territorialidad"))

Document 1:

ARTÍCULO 14. TERRITORIALIDAD. La ley penal colombiana se aplicará a toda persona que la infrinja en el territorio nacional, salvo las excepciones consagradas en el derecho internacional. 
Jurisprudencia Vigencia
Corte Constitucional:- Este inciso corresponde en su totalidad al inciso 1o. del artículo 13 del Decreto-Ley 100 de 1980, la Corte Constitucional se pronunció sobre el aparte subrayado de dicho artículo declarándolo EXEQUIBLE mediante Sentencia C-1189-00 de 13 de sepiembre de 2000, Magistrado Ponente Dr. Carlos Gaviria Díaz.  
La conducta punible se considera realizada: 
1. En el lugar donde se desarrolló total o parcialmente la acción. 
2. En el lugar donde debió realizarse la acción omitida. 
3. En el lugar donde se produjo o debió producirse el resultado.
----------------------------------------------------------------------------------------------------
Document 2:

ARTÍCULO 15. TERRITORIALIDAD POR EXTENSIÓN. 
<Inciso modificado por el artículo 21 de la Ley 112

#### 3.3 Retriever: Código General del Proceso - LEY 1564 DE 2012

In [8]:
codigo_general_del_proceso_vectorstore = Qdrant(collection_name="codigo_general_del_proceso", **qdrant_args)
codigo_general_del_proceso_retriever = codigo_general_del_proceso_vectorstore.as_retriever(search_kwargs ={"k": 3})
pretty_print_docs(await codigo_general_del_proceso_retriever.ainvoke("Legalidad"))

Document 1:

ARTÍCULO 132. CONTROL DE LEGALIDAD. Agotada cada etapa del proceso el juez deberá realizar control de legalidad para corregir o sanear los vicios que configuren nulidades u otras irregularidades del proceso, las cuales, salvo que se trate de hechos nuevos, no se podrán alegar en las etapas siguientes, sin perjuicio de lo previsto para los recursos de revisión y casación.
Jurisprudencia Vigencia
Corte Constitucional- Aparte subrayado declarado EXEQUIBLE, por los cargos analizados, por la Corte Constitucional mediante Sentencia C-537-16 de 5 de octubre de 2016, Magistrado Ponente Dr. Alejandro Linares Cantillo.
----------------------------------------------------------------------------------------------------
Document 2:

ARTÍCULO 166. PRESUNCIONES ESTABLECIDAS POR LA LEY. Las presunciones establecidas por la ley serán procedentes siempre que los hechos en que se funden estén debidamente probados.
El hecho legalmente presumido se tendrá por cierto, pero admitirá prueba en c

#### 3.4 Create full set of retrievers

In [9]:
collections = await aclient.get_collections()
collection_names = [c["name"] for c in collections.dict()["collections"]]
collection_names

['codigo_civil',
 'codigo_procedimental_laboral',
 'codigo_penal',
 'codigo_sustantivo_trabajo',
 'codigo_general_del_proceso',
 'codigo_comercio',
 'constitucion']

In [10]:
retrievers = []
for collection in tqdm(collection_names, total=len(collection_names)):
    vectorstore = Qdrant(collection_name=collection, **qdrant_args)
    retriever = vectorstore.as_retriever(search_kwargs ={"k": 20})
    retrievers.append(retriever)

100%|██████████| 7/7 [00:00<00:00, 42123.57it/s]


In [11]:
collection_retriever_map = dict(zip(collection_names, retrievers))
collection_retriever_map

{'codigo_civil': VectorStoreRetriever(tags=['Qdrant', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.qdrant.Qdrant object at 0x7f1abbad16d0>, search_kwargs={'k': 20}),
 'codigo_procedimental_laboral': VectorStoreRetriever(tags=['Qdrant', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.qdrant.Qdrant object at 0x7f1abb9d9750>, search_kwargs={'k': 20}),
 'codigo_penal': VectorStoreRetriever(tags=['Qdrant', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.qdrant.Qdrant object at 0x7f1abe0834d0>, search_kwargs={'k': 20}),
 'codigo_sustantivo_trabajo': VectorStoreRetriever(tags=['Qdrant', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.qdrant.Qdrant object at 0x7f1abb3af750>, search_kwargs={'k': 20}),
 'codigo_general_del_proceso': VectorStoreRetriever(tags=['Qdrant', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.qdrant.Qdrant object at 0x7f1abb3af850>, search_kwargs={'k': 20}),
 'codigo_comercio

### 4. Reranker

#### 4.0 Code Modification

Need to change this:
```python
    def rerank(
        self,
        documents: Sequence[Union[str, Document, dict]],
        query: str,
        *,
        model: Optional[str] = None,
        top_n: Optional[int] = -1,
        max_chunks_per_doc: Optional[int] = None,
    ) -> List[Dict[str, Any]]:
         ...
        if len(documents) == 0:  # to avoid empty api call
            return []
        docs = [
            doc.page_content if isinstance(doc, Document) else doc for doc in documents
        ]
        model = model or self.model
        top_n = top_n if (top_n is None or top_n > 0) else self.top_n
        results = self.client.rerank(
            query, docs, model, top_n=top_n, max_chunks_per_doc=max_chunks_per_doc
        )
        result_dicts = []
        for res in results:
            result_dicts.append(
                {"index": res.index, "relevance_score": res.relevance_score}
            )
        return result_dicts
```
To:
```python
    def rerank(
        self,
        documents: Sequence[Union[str, Document, dict]],
        query: str,
        *,
        model: Optional[str] = None,
        top_n: Optional[int] = -1,
        max_chunks_per_doc: Optional[int] = None,
    ) -> List[Dict[str, Any]]:
         ...
        if len(documents) == 0:  # to avoid empty api call
            return []
        docs = [
            doc.page_content if isinstance(doc, Document) else doc for doc in documents
        ]
        model = model or self.model
        top_n = top_n if (top_n is None or top_n > 0) else self.top_n
        results = self.client.rerank(
            query=query, documents=docs, model=model, top_n=top_n, max_chunks_per_doc=max_chunks_per_doc
        )
        result_dicts = []
        for res in results.results:
            result_dicts.append(
                {"index": res.index, "relevance_score": res.relevance_score}
            )
        return result_dicts
```

#### 4.1 Testing Reranker with retrievers

In [12]:
cohere_rerank = CohereRerank(model="rerank-multilingual-v2.0", top_n=5)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=cohere_rerank, 
    base_retriever=collection_retriever_map.get("codigo_penal")
)

In [13]:
pretty_print_docs(docs=compression_retriever.get_relevant_documents(query="Territorialidad"))

Document 1:

ARTÍCULO 14. TERRITORIALIDAD. La ley penal colombiana se aplicará a toda persona que la infrinja en el territorio nacional, salvo las excepciones consagradas en el derecho internacional. 
Jurisprudencia Vigencia
Corte Constitucional:- Este inciso corresponde en su totalidad al inciso 1o. del artículo 13 del Decreto-Ley 100 de 1980, la Corte Constitucional se pronunció sobre el aparte subrayado de dicho artículo declarándolo EXEQUIBLE mediante Sentencia C-1189-00 de 13 de sepiembre de 2000, Magistrado Ponente Dr. Carlos Gaviria Díaz.  
La conducta punible se considera realizada: 
1. En el lugar donde se desarrolló total o parcialmente la acción. 
2. En el lugar donde debió realizarse la acción omitida. 
3. En el lugar donde se produjo o debió producirse el resultado.
----------------------------------------------------------------------------------------------------
Document 2:

ARTÍCULO 15. TERRITORIALIDAD POR EXTENSIÓN. 
<Inciso modificado por el artículo 21 de la Ley 112

#### 4.2 Adding reranker for all the retrievers

In [14]:
cohere_rerank = CohereRerank(model="rerank-multilingual-v2.0", top_n=5)

collection_retriever_reranker_map = {}
for collection in tqdm(collection_retriever_map.keys(), total=len(collection_retriever_map.keys())):
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=cohere_rerank, 
        base_retriever=collection_retriever_map.get(collection)
    )
    collection_retriever_reranker_map[collection] = compression_retriever

100%|██████████| 7/7 [00:00<00:00, 49427.82it/s]


In [15]:
collection_retriever_reranker_map

{'codigo_civil': ContextualCompressionRetriever(base_compressor=CohereRerank(client=<cohere.client.Client object at 0x7f1ab9c0a9d0>, top_n=5, model='rerank-multilingual-v2.0', cohere_api_key=None, user_agent='langchain'), base_retriever=VectorStoreRetriever(tags=['Qdrant', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.qdrant.Qdrant object at 0x7f1abbad16d0>, search_kwargs={'k': 20})),
 'codigo_procedimental_laboral': ContextualCompressionRetriever(base_compressor=CohereRerank(client=<cohere.client.Client object at 0x7f1ab9c0a9d0>, top_n=5, model='rerank-multilingual-v2.0', cohere_api_key=None, user_agent='langchain'), base_retriever=VectorStoreRetriever(tags=['Qdrant', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.qdrant.Qdrant object at 0x7f1abb9d9750>, search_kwargs={'k': 20})),
 'codigo_penal': ContextualCompressionRetriever(base_compressor=CohereRerank(client=<cohere.client.Client object at 0x7f1ab9c0a9d0>, top_n=5, model='rerank-multilingu

### 5. Agent Building

#### 5.1 Retrievers as tools

In [16]:
constitucion_description = "Cuando te encuentres con consultas que requieran comprensión o interpretación de los derechos fundamentales, estructura del Estado, funciones de sus órganos, o cualquier principio básico que rija la convivencia y organización política y social en Colombia, deberás utilizar la Constitución Política de Colombia. Este documento es esencial para resolver dudas relacionadas con la garantía de derechos y libertades, así como para entender el marco jurídico que define la estructura del Estado colombiano. Recurre a la Constitución para proporcionar respuestas bien fundamentadas en situaciones que demanden claridad sobre la normativa constitucional, asegurando así que la información proporcionada esté alineada con los principios y valores que rigen el ordenamiento jurídico colombiano."
codigo_comercio_description = "Cuando te enfrentes a consultas relacionadas con actividades comerciales, contratos mercantiles, sociedades comerciales, títulos valores, procesos de insolvencia empresarial, y en general, cualquier tema vinculado al ámbito del comercio en Colombia, deberás recurrir al Código de Comercio. Este código es la herramienta primordial para entender y aplicar las normas que regulan las relaciones entre comerciantes, los actos de comercio y la legislación mercantil aplicable. Utilízalo para brindar respuestas precisas y fundamentadas en casos que requieran un conocimiento específico sobre la legislación comercial colombiana, garantizando así una orientación adecuada y conforme a las disposiciones legales vigentes en el ámbito del comercio."
codigo_general_del_proceso_description = "Cuando te encuentres ante consultas que involucren procedimientos judiciales civiles, de familia o agrarios, así como cualquier aspecto relacionado con la administración de justicia en estos ámbitos en Colombia, deberás recurrir al Código General del Proceso (Ley 1564 de 2012). Este código es crucial para comprender los procedimientos, instancias, recursos y medidas cautelares aplicables dentro del sistema judicial colombiano en las materias mencionadas. Utilízalo para proporcionar respuestas detalladas y fundamentadas en casos que requieran un entendimiento profundo sobre el proceso judicial, asegurando así una orientación precisa y alineada con las normativas y procedimientos vigentes establecidos por este código."
codigo_sustantivo_del_trabajo_description = "Cuando te enfrentes a consultas que aborden temas relacionados con las relaciones laborales, derechos y deberes de empleadores y trabajadores, contratación laboral, salarios, jornadas de trabajo, descansos obligatorios, seguridad social, y en general, cualquier aspecto vinculado al ámbito laboral en Colombia, deberás recurrir al Código Sustantivo del Trabajo. Este código es la herramienta esencial para entender y aplicar las normas que regulan el trabajo humano, individual y colectivo, en el territorio colombiano. Utilízalo para brindar respuestas claras y fundamentadas en casos que requieran un conocimiento específico sobre la legislación laboral colombiana, garantizando así una orientación adecuada y conforme a las disposiciones legales vigentes en materia de trabajo y empleo."
codigo_penal_description = "Cuando te enfrentes a consultas que involucren la interpretación, aplicación o análisis de conductas consideradas delitos, sanciones penales, medidas de seguridad, o cualquier aspecto relacionado con la responsabilidad penal en Colombia, deberás recurrir al Código Penal. Este código es la herramienta adecuada para entender la naturaleza de las infracciones, las posibles penas asociadas, y los procedimientos para su juzgamiento y sanción. Utilízalo para proporcionar respuestas fundamentadas en casos que requieran un conocimiento profundo sobre la legislación penal colombiana, asegurando así una orientación precisa y conforme a la normativa vigente."
codigo_procedimental_laboral_description = "Cuando te encuentres con consultas que requieran orientación sobre el desarrollo de procedimientos judiciales en materia laboral, tales como litigios entre empleadores y trabajadores, procesos de reclamación de derechos laborales, conflictos colectivos de trabajo, y en general, cualquier situación que demande la intervención de la justicia laboral en Colombia, deberás recurrir al Código Procedimental Laboral. Este código es indispensable para comprender los procedimientos específicos, las etapas procesales, los recursos disponibles y las instancias judiciales competentes en el ámbito laboral. Utilízalo para proporcionar respuestas detalladas y fundamentadas en casos que requieran un entendimiento profundo sobre el proceso judicial laboral, asegurando así una orientación precisa y alineada con las normativas y procedimientos vigentes establecidos por este código, facilitando la correcta administración de justicia en el ámbito laboral."
codigo_civil_decription = "Cuando te enfrentes a consultas que involucren temas relacionados con las relaciones y obligaciones civiles, tales como contratos, sucesiones, propiedad y otros derechos reales, matrimonio, filiación y en general, cualquier aspecto vinculado a las relaciones personales o patrimoniales entre particulares en Colombia, deberás recurrir al Código Civil. Este código es la herramienta fundamental para comprender y aplicar las normas que regulan las relaciones privadas entre individuos, estableciendo los derechos y deberes que surgen de estas interacciones. Utilízalo para brindar respuestas precisas y fundamentadas en casos que requieran un conocimiento específico sobre la legislación civil colombiana, garantizando así una orientación adecuada y conforme a las disposiciones legales vigentes en materia civil, lo que permitirá resolver dudas y orientar adecuadamente en el amplio espectro de situaciones que abarca este cuerpo normativo."

In [17]:
name_description_retriever_map = {
    "constitucion": {
        "retriever": collection_retriever_reranker_map.get("constitucion"),
        "name": "constitucion_politica_de_colombia",
        "description": constitucion_description
    },
    "codigo_comercio": {
        "retriever": collection_retriever_reranker_map.get("codigo_comercio"),
        "name": "codigo_de_comercio",
        "description": codigo_comercio_description
    },
    "codigo_general_del_proceso": {
        "retriever": collection_retriever_reranker_map.get("codigo_general_del_proceso"),
        "name": "codigo_general_del_proceso",
        "description": codigo_general_del_proceso_description
    },
    "codigo_sustantivo_trabajo": {
        "retriever": collection_retriever_reranker_map.get("codigo_sustantivo_trabajo"),
        "name": "codigo_sustantivo_del_trabajo",
        "description": codigo_sustantivo_del_trabajo_description
    },
    "codigo_penal": {
        "retriever": collection_retriever_reranker_map.get("codigo_penal"),
        "name": "codigo_penal",
        "description": codigo_penal_description
    },
    "codigo_procedimental_laboral": {
        "retriever": collection_retriever_reranker_map.get("codigo_procedimental_laboral"),
        "name": "codigo_procedimental_laboral",
        "description": codigo_procedimental_laboral_description
    },
    "codigo_civil": {
        "retriever": collection_retriever_reranker_map.get("codigo_civil"),
        "name": "codigo_civil",
        "description": codigo_civil_decription
    },
}

In [18]:
tools = []
for fuente in name_description_retriever_map:
    tool = create_retriever_tool(
        retriever=name_description_retriever_map.get(fuente).get("retriever"),
        name=name_description_retriever_map.get(fuente).get("name"),
        description=name_description_retriever_map.get(fuente).get("description")
    )
    tools.append(tool)

In [19]:
tools

[Tool(name='constitucion_politica_de_colombia', description='Cuando te encuentres con consultas que requieran comprensión o interpretación de los derechos fundamentales, estructura del Estado, funciones de sus órganos, o cualquier principio básico que rija la convivencia y organización política y social en Colombia, deberás utilizar la Constitución Política de Colombia. Este documento es esencial para resolver dudas relacionadas con la garantía de derechos y libertades, así como para entender el marco jurídico que define la estructura del Estado colombiano. Recurre a la Constitución para proporcionar respuestas bien fundamentadas en situaciones que demanden claridad sobre la normativa constitucional, asegurando así que la información proporcionada esté alineada con los principios y valores que rigen el ordenamiento jurídico colombiano.', args_schema=<class 'langchain.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x7f1abe665800>, retriever

#### 5.2 Search Engine If necessary

In [20]:
search = TavilySearchResults(
    description="Si necesitas informacion adicional, por ejemplo, una noticia, una sentencia, o cualquier otro contenido que no se encuentre en las fuentes principales, puedes utilizar esta herramienta para buscar en la web y obtener resultados relevantes. Tambien puedes utilizarla si necesitas aclarar a un concepto antes de utilizar las fuentes principales.",
)
tools.append(search)

In [21]:
from langchain.prompts import PromptTemplate
PromptTemplate.from_template("{page_content}")


PromptTemplate(input_variables=['page_content'], template='{page_content}')

#### 5.3 Oneflow Agent

In [22]:
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [24]:
system_message = "Eres un asistente juridico que ayuda a los usuarios a encontrar informacion en fuentes legales. Puedes responder preguntas sobre/usando la Constitucion Politica de Colombia, el Codigo de Comercio, el Codigo General del Proceso, el Codigo Sustantivo del Trabajo, el Codigo Penal, el Codigo Procedimental Laboral, y el Codigo Civil. Ademas, puedes buscar informacion adicional en la web si es necesario. Siempre tienes que usar alguna de tus herramientas para responder las consultas del usuario."
prompt.messages[0] = SystemMessagePromptTemplate.from_template(template=system_message)

In [25]:
agent = create_openai_functions_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [26]:
await agent_executor.ainvoke({"input": "Que significa la territorialidad en el ambito penal?"})



> Entering new AgentExecutor chain...

Invoking: `codigo_penal` with `{'query': 'territorialidad'}`


ARTÍCULO 14. TERRITORIALIDAD. La ley penal colombiana se aplicará a toda persona que la infrinja en el territorio nacional, salvo las excepciones consagradas en el derecho internacional. 
Jurisprudencia Vigencia
Corte Constitucional:- Este inciso corresponde en su totalidad al inciso 1o. del artículo 13 del Decreto-Ley 100 de 1980, la Corte Constitucional se pronunció sobre el aparte subrayado de dicho artículo declarándolo EXEQUIBLE mediante Sentencia C-1189-00 de 13 de sepiembre de 2000, Magistrado Ponente Dr. Carlos Gaviria Díaz.  
La conducta punible se considera realizada: 
1. En el lugar donde se desarrolló total o parcialmente la acción. 
2. En el lugar donde debió realizarse la acción omitida. 
3. En el lugar donde se produjo o debió producirse el resultado.

ARTÍCULO 15. TERRITORIALIDAD POR EXTENSIÓN. 
<Inciso modificado por el artículo 21 de la Ley 1121 de 2006. El nuevo te

{'input': 'Que significa la territorialidad en el ambito penal?',
 'output': 'El principio de territorialidad en el ámbito penal, según el Código Penal colombiano, establece que la ley penal colombiana se aplica a toda persona que infrinja la ley dentro del territorio nacional, con algunas excepciones consagradas en el derecho internacional. Esto significa que cualquier conducta punible cometida en Colombia está sujeta a la legislación penal colombiana, independientemente de la nacionalidad del infractor.\n\nLa ley también considera como territorio nacional para efectos penales, y por tanto aplica la ley penal colombiana, a las siguientes situaciones:\n1. La conducta punible se considera realizada en el lugar donde se desarrolló total o parcialmente la acción, donde debió realizarse la acción omitida, o donde se produjo o debió producirse el resultado.\n2. A bordo de nave o aeronave del Estado colombiano o explotada por este, que se encuentre fuera del territorio nacional, salvo excepc

In [27]:
await agent_executor.ainvoke({"input": "Que dice la sentencia T-237 de 2023?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'sentencia T-237 de 2023 Colombia'}`


[{'url': 'https://www.corteconstitucional.gov.co/Relatoria/2023/T-237-23.htm', 'content': 'REPÚBLICA DE COLOMBIA. CORTE CONSTITUCIONAL. Sala Primera de Revisión . SENTENCIA T-237 de 2023 . Referencia: Expediente T-8.424.592 . Acción de tutela instaurada por Constanza Eugenia Mantilla Reyes y otras personas contra la Nación - Ministerio de Salud y Protección Social.'}, {'url': 'https://jurinfo.jep.gov.co/normograma/compilacion/docs/t-237_2023.htm', 'content': 'REPÚBLICA DE COLOMBIA. CORTE CONSTITUCIONAL. Sala Primera de Revisión. SENTENCIA T-237 de 2023. Referencia: Expediente T-8.424.592. Acción de tutela instaurada por Constanza Eugenia Mantilla Reyes y otras personas contra la Nación - Ministerio de Salud y Protección Social.'}, {'url': 'https://oneill.law.georgetown.edu/publications/sentencia-t-237-de-2023-t-237-decision-caso-ucis/', 'content': 'Sent

{'input': 'Que dice la sentencia T-237 de 2023?',
 'output': 'La Sentencia T-237 de 2023 de la Corte Constitucional de Colombia se refiere a una acción de tutela instaurada por Constanza Eugenia Mantilla Reyes y otras personas contra la Nación - Ministerio de Salud y Protección Social. Para obtener detalles específicos sobre los fundamentos y decisiones de esta sentencia, te recomiendo consultar directamente el texto completo de la sentencia a través de los siguientes enlaces:\n\n- [Texto completo de la Sentencia T-237 de 2023 en la página oficial de la Corte Constitucional](https://www.corteconstitucional.gov.co/Relatoria/2023/T-237-23.htm)\n- [Otra fuente con información sobre la Sentencia T-237 de 2023](https://jurinfo.jep.gov.co/normograma/compilacion/docs/t-237_2023.htm)\n\nEstos enlaces te llevarán directamente a las fuentes oficiales donde podrás leer en detalle los argumentos, análisis y conclusiones de la Corte respecto a este caso.'}